---

# Code pour les flags 21 à 23



---

# Flag 21 : 
tme.pratt TME 3 
(se situe tour 15 au 3e étage ! )

On va dans une tour avec un ascenseur au niveau SB et on tappe #!sudo puis ascenseur. On monte au niveau JU et on se dirige vers la tour 15 au 3e étage. Le résultat du programme est sauvegardé dans un fichier json : "flag21_tme3.pratt.json"


In [4]:
import random
from sympy import factorint
import json

a = int("5ce64296d948e90bbc2fc517fcf2efb644dc1889e21bae8a527d3ebbf301ad93fb0e348ffb89d22ab02674d7774e170db7555847984dd55719c8ceee97f25fe853717201d33b71e8157af54eb1bda21d74cec2b4ebc4e3f6ee93c87fbe5b2a4cb56c8fd185aa2ae24f47dfe630990219891fe974e0b096f4d7f5a4cb9a909580", 16)
b = a + 2**960

def produit_liste(liste_nombres):
    resultat = 1
    for nombre in liste_nombres:
        resultat *= nombre
    return resultat

# Test de primalité de Miller-Rabin
def miller_rabin(n, iterations):
    if (n == 2):
        return True
    if (n % 2 == 0 or n < 2):
        return False

    r = 0
    s = n - 1
    while (s % 2 == 0):
        r += 1
        s //= 2

    compteur_test = 0
    while (compteur_test < iterations):
        base = random.randrange(2, n - 1)
        x = pow(base, s, n)
        # print(f"base={base}, x={x}")
        if (x != 1 and x != n - 1):
            repetition = 0
            while ((repetition < r - 1) and (x != n - 1)):
                x = pow(x, 2, n)
                repetition += 1
            if (x != n - 1):
                return False
        compteur_test += 1

    return True

def taille_en_octets(entier):
    # print(f"taille_en_octets de {entier} = {(entier.bit_length() + 7) // 8}")
    return (entier.bit_length() + 7) // 8

def trouve_generateur(prime, diviseurs):
    while True:
        candidat = random.randrange(2, prime - 1)
        # print(f"candidat générateur = {candidat}")
        if all(pow(candidat, (prime - 1)//d, prime) != 1 for d in diviseurs):
            return candidat

def construire_certificat(diviseurs_certif):
    certificats = []
    for diviseur in diviseurs_certif:
        cert = {'p': diviseur}
        if diviseur >= 1024:
            facteurs = factorint(diviseur - 1)
            sous_divs = []
            for facteur, exposant in facteurs.items():
                sous_divs.extend([facteur] * exposant)
            # print(f"facteurs de {diviseur - 1} = {facteurs}, sous_divs = {sous_divs}")
            cert['g'] = trouve_generateur(diviseur, sous_divs)
            cert['pm1'] = construire_certificat(sous_divs)
        certificats.append(cert)
    return certificats

liste_pk = [2]
Q = 2
constante_c = 20000
taille_b = taille_en_octets(b)
Q_max = (b - a) // (constante_c * taille_b**2)

# print(f"Q_max = {Q_max}")
while taille_en_octets(Q_max) != taille_en_octets(Q):
    premier_candidat = random.randrange(2, min(pow(2, 160), pow(2, 8 * (taille_en_octets(Q_max) - taille_en_octets(Q)))))
    # print(f"premier_candidat = {premier_candidat}")
    if miller_rabin(premier_candidat, 10):
        Q *= premier_candidat
        liste_pk.append(premier_candidat)

p_k = random.randrange(a // Q, b // Q)
while not miller_rabin(p_k, 1) or not miller_rabin(Q * p_k + 1, 1):
    p_k = random.randrange(a // Q, b // Q)

liste_pk.append(p_k)
grand_p = produit_liste(liste_pk) + 1
generateur = trouve_generateur(grand_p, liste_pk)

def convertir_objet_int(objet):
    if isinstance(objet, dict):
        return {k: convertir_objet_int(v) for k, v in objet.items()}
    elif isinstance(objet, list):
        return [convertir_objet_int(i) for i in objet]
    elif hasattr(objet, '__int__'):
        try:
            return int(objet)
        except:
            return objet
    else:
        return objet

certificat_pratt = {'p': grand_p, 'g': generateur, 'pm1': construire_certificat(liste_pk)}

# print(f"certificat_pratt = {certificat_pratt}")
with open('flag21_tme3.pratt.json', 'w') as fichier:
    json.dump(convertir_objet_int(certificat_pratt), fichier)

print("terminé")


terminé


# FLAG 22 
crc64.preimage Atrium 1er étage.

Le principe du flag est de trouver une préimage pour la fonction CRC64-iso de la signature du mdp de "MKT01". Dans un premier temps on récupère l'empreinte du mdp sur le terminal en Recovery Mode. On effectue ensuite une attaque sur la fonction CRC : l'empreinte S(x) correspond au reste de la division de x⁶⁴ * P(x) par un polynôme irréductible Q(x). Comme Q(x) est irréductible sur GF(2), on peut travailler dans le corps fini GF(2⁶⁴), et ainsi inverser l'opération en multipliant S(x) par l'inverse de x⁶⁴ modulo Q(x).

Une fois la préimage trouvée, on peut se connecter en tant que "MKT01" sur le terminal en Secure Mode.

In [1]:
from binascii import unhexlify, hexlify

truc = 0x1000000000000001b
val = int.from_bytes(unhexlify("52052ce33bd16914"), 'big')
gen_poly = 0b11011

def gf2_mul(x, y):
    out = 0
    while y:
        if y & 1:
            out ^= x
        x = x << 1
        y = y >> 1
    return out

def mod(x):
    while x.bit_length() > 64:
        n = x.bit_length() - 65
        x = x ^ (truc << n)
    return x

def inverse(a):
    u = a
    v = truc
    a1 = 1
    a2 = 0
    while u != 1:
        j = u.bit_length() - v.bit_length()
        if j < 0:
            u, v = v, u
            a1, a2 = a2, a1
            j = -j
        u = u ^ (v << j)
        a1 = a1 ^ (a2 << j)
        # print(f"u = {u}, v = {v}, a1 = {a1}, a2 = {a2}, j = {j}")
    return mod(a1)

inv = inverse(gen_poly)
# print(f"Inverse of gen_poly = {inv}")

final = mod(gf2_mul(inv, val))
# print(f"Multiplication result = {gf2_mul(inv, val)}")
# print(f"Final result before mod = {final}")

res = final.to_bytes((final.bit_length() + 7) // 8, 'big')
# print(f"Result as bytes = {res}")

print("Préimage :", hexlify(res).decode())


Préimage : b0388d2e1ef70463


# FLAG 23 : Robot gardien Ciscu 

ATTENTION le flag n'a pas été réussi, j'ai l'erreur : " | The signature must be hex-encoded |" alors que la signature que je donne au robot gardien est bien en hexa

L'idée du flag est de générer sans l'accord direct du directeur, une signature disant que j'ai droit de prendre ce foutu bibliodrone alors que j'ai seulement demandé au directeur une signature d'un message bizarre qui n'avait pas de sens (à priori...). Pour cela, on a besoin de la clé publique du directeur qui est disponible dans le laptop de l'auditorium du CICSU. On a aussi besoin de demander une signature du directeur sur un message bien choisi depuis le laptop de l'auditorium.

On copie cette signature dans la 2e partie du programme ci-dessous et on obtient la signature du message qu'on souhaitait de base. Finalement, on donne cette signature au robot gardien et il est censé nous donner le biblio drone et un flag.



In [2]:
from hashlib import sha256
from Crypto.PublicKey import RSA

msg = b"I, the lab director, hereby grant poupeaup permission to take the BiblioDrone-NG."

HASH_ID = b'\x30\x31\x30\r\x06\t`\x86H\x01e\x03\x04\x02\x01\x05\x00\x04 '

def entierVersOctets(valeur: int, taille: int) -> bytes:
    return valeur.to_bytes(taille, 'big')

def octetsVersEntier(data: bytes) -> int:
    return int.from_bytes(data, 'big')

def encoderMessagePKCS1(msg: bytes, longueur: int) -> bytes:
    hash_val = sha256(msg).digest()
    info = HASH_ID + hash_val

    # print(f"Hash : {hash_val.hex()}") 
    # print(f"Info (Hash ID + Hash) : {info.hex()}") 

    if len(info) + 11 > longueur:
        raise ValueError("Message trop long")

    padding = b'\xff' * (longueur - len(info) - 3)
    # print(f"Padding : {padding.hex()}") 
    return b'\x00\x01' + padding + b'\x00' + info

def decoderMessagePKCS1(msgCode: bytes, longueur: int) -> bytes:
    if len(msgCode) != longueur:
        raise ValueError("Taille invalide")
    if not msgCode.startswith(b'\x00\x01'):
        raise ValueError("Header invalide")
    try:
        separateur = msgCode.index(b'\x00', 2)
    except ValueError:
        raise ValueError("Séparateur manquant")
    if separateur < 10:
        raise ValueError("Padding insuffisant")
    info = msgCode[separateur+1:]
    if not info.startswith(HASH_ID):
        raise ValueError("Hash ID invalide")
    return info[len(HASH_ID):]

def tailleCleOctets(n: int) -> int:
    return (n.bit_length() + 7) // 8

def signerRSA(n: int, d: int, msg: bytes) -> bytes:
    k = tailleCleOctets(n)
    enc = encoderMessagePKCS1(msg, k)
    m = octetsVersEntier(enc)
    s = pow(m, d, n)
    return entierVersOctets(s, k)

def verifierRSA(n: int, e: int, msg: bytes, signature: bytes) -> bool:
    k = tailleCleOctets(n)
    if len(signature) != k:
        raise ValueError("Longueur signature invalide")
    s = octetsVersEntier(signature)
    m = pow(s, e, n)
    enc = entierVersOctets(m, k)
    try:
        h = decoderMessagePKCS1(enc, k)
        return h == sha256(msg).digest()
    except ValueError:
        return False

x = 7

with open("keys/pkey_director.pem", "rb") as fichier:
    clePublique = RSA.import_key(fichier.read())

# Affichage de la taille de la clé publique
# print(f"Clé publique taille (octets) : {clePublique.size_in_bytes()}") 

msgEncode = encoderMessagePKCS1(msg, clePublique.size_in_bytes())
# print(f"Message encodé (PKCS1) : {msgEncode.hex()}") 

entierMsg = octetsVersEntier(msgEncode)
# print(f"Message en entier : {entierMsg}") 

msgTransforme = (entierMsg * pow(x, clePublique.e, clePublique.n)) % clePublique.n
# print(f"Message transformé : {msgTransforme:x}") 

print("Message à faire signer :")
print(f"{msgTransforme:x}")

xInverse = pow(x, -1, clePublique.n)
# print(f"Inverse de x : {xInverse}") 


Message à faire signer :
679b7bedd7d74f07e086e09bcb1521742ec8fb4811909be73df3a96396179e7b3ee65917cf4595d60d90a9292cf69d4bcd5772d6dddd71a4f5a81e3059f600edb7644c4bfb75042e4cdb395f736049ab30ea2c5c21f498f16d634f5288d784581d3b867c2d8face274a57d8b904aa01a93e7a8113e10cc05bb65d5dcdacdba97a8dcd26125d38b197a1eaf17d5e4e2e90bcd0c0093c719627322304044a5350833f5f2e0b2f508fd6a6601af1f340f1871859467ecb1f14c5375a55501847d1b7b65524930e9e913d080eb04287c952edd41475feeb238c96d87cc5289daa11eb8189b049d50984ef38df070090832c2b93b879dd9c58cf377d54192e7e14c79


In [3]:
sign_recup= int("29d167213625d467b34a1e64bf3bd0008340f4775c96a681a04f5204d3695a88a1324f63df92c29f460c15b66e179e74232a53832eee14738c40f10f056b029aa282e0a15869cff325cc1c085d6a6635467120995f22f9c8e1dd77ad73c97cc100e7abf40c5c6bc5ce528aafb6a07fc5cae614371048935e005ed220578bc9f7ec76a640c19810b547f081b60d97a0faafbcaafff2adddcf65a2257e48482e6940c366534df72b2b4353c60b0ef8da43b3a534f035f607a100ecc387afbde17c04a29aec8c07391c893d3327b531d0db67cbc3599d30249bcb68aee248a2c48fe1250ffbb2ac16b7557a0528508733e30f359b6c35951ec78e2de3546e335efc", 16)
sign_souhait=(xInverse*sign_recup)%clePublique.n
print("la signature qu'on souhaitait : ",f"{sign_souhait:x}")

la signature qu'on souhaitait :  5f957e02c4e8c0ed0784d7c1b51b00012c022ec7af0f33716e6c292f9a1561384be0b576914f73b53264c3ea2035fc772bce75be22202ebef76fddd930f4983ce12b2603133affe29f8964a567cea079c594dccc472b5f8204435ab108a7f894945aad9b89f7d1c420bcaab633b7ff7af45709a26e5cbe8db7b4296e7ef65fed8a33c5264cc94ac2ed9371a01f116ff3dad3cfffe18d68b57a973120a5374582dd2c5799d6c73e19be7656f4b481f2e3e3c2c2250da01170021d2ca3dad695adc173ab414010828a5e42be119e28b8d0ed3f755f1e24e5f663386b29ca989cb694e6fff62aadeac79ecdc2a54a599b2b907a87d2c39dfd35d74450c0fbe3224
